In [2]:
import cv2
from ISR.models import RDN, RRDN
from PIL import Image
import numpy as np
from time import gmtime, strftime
from pathlib import Path

Using TensorFlow backend.


## Params model and image paths

In [18]:
# Params model and path pre-train weights
weights = '../../weights/rrdn-C4-D3-G32-G032-T10-x4/2020-01-04_22_04_54/rrdn-C4-D3-G32-G032-T10-x4batch000100_epoch092.hdf5'
C = 4
D = 3
G = 32
G0 = 32
T = 10
scale = 4
dpi = 300
type = 'RRDN'

# Image paths
time = strftime("%Y-%m-%d_%H_%M_%S", gmtime())
path = '../../Results/Source/'
path_sr = './'
name_model = 'sr_{type}_{c}_{d}_{g}_{g0}_{t}_{scale}_{time}'

lr = str(Path(path + '75.jpg'))
sr = str(Path(path_sr + name_model.format(type=type, c=C, d=D, g=G, g0=G0, t=T, scale=scale, time=time) + '.jpg'))
hr = str(Path(path + '300.jpg'))
dst_path = str(Path(path_sr + 
                    name_model.format(type=type, c=C, d=D, g=G, g0=G0, t=T, scale=scale, time=time) + 
                    '_mask_{mask}.jpg'))

masks = {'full': [1, 1, 1],
         'r':    [1, 0, 0],
         'g':    [0, 1, 0],
         'b':    [0, 0, 1]}

## Load model and predict

In [13]:
%%time
if type is 'RDN':
    rdn = RDN(arch_params={'C': C, 'D': D, 'G': G, 'G0': G0, 'T': T, 'x': scale})
else:
    rdn = RRDN(arch_params={'C': C, 'D': D, 'G': G, 'G0': G0, 'T': T, 'x': scale})

rdn.model.load_weights(weights)
image_sr = rdn.predict(np.array(Image.open(lr)))
Image.fromarray(image_sr).save(sr, dpi=(dpi, dpi))


Wall time: 1min 30s


## Load images

In [16]:
image_HR = cv2.imread(hr)
image_SR = cv2.imread(sr)

if image_HR.size > image_SR.size:
    image_HR = image_HR[:image_SR.shape[0], :image_SR.shape[1]]
else:
    image_SR = image_SR[:image_HR.shape[0], :image_HR.shape[1]]



## Masking the original image with the difference with the predicted 

In [19]:
for mask_name, mask in masks.items():
    mask_div = np.ones(image_HR.shape) * 255 - image_HR + image_SR
    mask_div = mask_div.transpose([2, 0, 1])
    mask_div = np.array([mask_div[i] * mask[i] for i in range(3)], dtype=np.uint8)
    mask_div = mask_div.transpose([1, 2, 0])
    Image.fromarray(mask_div + image_HR).save(dst_path.format(mask=mask_name, time=time))